In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import arviz as az
from time import time

import pyro
import torch
import pyro.optim as optim
from pyro.infer import SVI, JitTrace_ELBO, Predictive
from pyro.contrib.autoguide import AutoDiagonalNormal

from data_proc import load_raw_data, processData, transform_data, split_and_prep_data
from subreddit_model import subreddit_model, subreddit_guide
from model_utils import get_y_pred, get_type_only_y_pred, get_s_means, get_r_means, get_t_means, get_mean_y_pred
from inference import run_svi, plot_losses
from evaluation import plot_predictions, get_samples, gather_az_inference_data, plot_pp_pdf, plot_pp_cdf, plot_ppc, plot_residuals


pyro.enable_validation(True)

# Complete Level Model
Here, we will model each post as coming from one of four types, one subreddit, and being about one story.

# Data

First, we load the Reddit datasets

In [2]:
comments, corrections, news = load_raw_data()

### Process and Unpack Data

In [3]:
COMMENTS_ONLY = True

In [4]:
data_tuple, lookup_tuple, label_tuple = processData(news + corrections, comments, comments_only=COMMENTS_ONLY, minutes=60)

In [5]:
original_p_data, t_data, s_data, r_data, y = data_tuple
p_types, p_stories, p_subreddits = lookup_tuple
countries, authors, story_claim_titles, subreddits = label_tuple

p_data variables (in order):

0. bias
1. num_comments in first _ mins
2. num_subscribers at time of submission

types

0. Real News
1. Fake News
2. Correction on Real News
3. Correction on Fake News

t_data variables (in order):

0. bias
1. Real News Dummy
2. Fake News Dummy
3. Correction on Real News Dummy
4. Correction on Fake News Dummy

s_data variables:

0. bias
1. country dummies (multiple may be "on")
2. author dummies

r_data variables:

0. bias
1. subreddit dummies

### Data Transformations

In [6]:
# before we change anything, save the original p_data
p_data = transform_data(original_p_data)

In [7]:
# Split into Test and Train datasets
train_data, test_data, unsplit_data = split_and_prep_data(p_data, t_data, s_data, r_data, y, p_types, p_stories, p_subreddits, train_frac=0.7)
p_data_train, y_train, p_types_train, p_stories_train, p_subreddits_train = train_data
p_data_test, y_test, p_types_test, p_stories_test, p_subreddits_test = test_data
t_data, s_data, r_data = unsplit_data

-----

# Modeling

2 Regressions.

Post-Level Regression

y_pt = phi_0,t * bias + phi_1,t * first_hour_comments + epsilon_pt

Type-Level Regressions (2)

phi_0t = eta_00 * bias + eta_01 silliness_t + nu_0t

phi_1t = eta_10 * bias + eta_11 silliness_t + nu_1t

In [8]:
model = subreddit_model
guide = subreddit_guide

# Inference

## SVI

In [ ]:
svi, losses = run_svi(model, guide, train_data, unsplit_data, subsample=False, num_iters=5000, lr=1e-2, zero_inflated=torch.Tensor([False]))

/Users/marcusdaly/Documents/Grad/Y1S1/MLWithProbProg/Probabilistic-Programming/model/subreddit_model.py:196: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if zero_inflated[0]:
/Users/marcusdaly/Documents/Grad/Y1S1/MLWithProbProg/Probabilistic-Programming/model/subreddit_model.py:62: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if zero_inflated[0]:
/Users/marcusdaly/Documents/Grad/Y1S1/MLWithProbProg/Probabilistic-Programming/model/subreddit_model.py:100: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We c

In [ ]:
plot_losses(losses, log_scale = True, skip_first = 0)

----

# Evaluation

Recall:

Variables (in order):

0. num_comments
1. type
2. num_comments in first _ mins
3. num_subscribers at submission time

types

0. Real News
1. Fake News
2. Correction on Real News
3. Correction on Fake News

t level Variables (in order):

0. Is Review? (vs. news)
1. Is about fake story? (Not included yet)

Check the fit

2 Regressions.

Post-Level Regression

y_pt = mu + phi_0,t * bias + phi_1,t * first_hour_comments + epsilon_pt

Type-Level Regressions (2)

phi_0t = eta_00 * bias + nu_0t

phi_1t = eta_10 * bias + nu_1t

In [ ]:
INDEP = 1

In [ ]:
# get the means to be able to plot smoothly with just type data
# and for better interpretation
t_means = get_t_means(torch.Tensor(p_types).long(), t_data)
r_means = get_r_means(torch.Tensor(p_subreddits).long(), r_data)

In [ ]:
# Only using types (Smooth)
if COMMENTS_ONLY:
    y_pred_type_only = get_type_only_y_pred(p_data=p_data, t_data=t_data, p_types=p_types, r_means=r_means)
    for i in range(4):
        # actual predictions, using just type data
        plot_predictions(original_p_data, y, p_types, original_p_data, y_pred_type_only, p_types, indep=INDEP, only_type=i, log_scale=True, alpha=0.1)

In [ ]:
y_pred_mean = get_mean_y_pred(p_data=p_data, t_means=t_means, r_means=r_means)
plot_predictions(original_p_data, y, p_types, original_p_data, y_pred_mean, p_types, indep=INDEP, log_scale=True)

In [ ]:
# Using all data (Jagged—other factors at play)
y_pred = get_y_pred(p_data=p_data, t_data=t_data, r_data=r_data, p_types=p_types, p_subreddits=p_subreddits, s_data=None, p_stories=None)

for i in range(4):
    plot_predictions(original_p_data, y, p_types, original_p_data, y_pred, p_types, indep=INDEP, only_type=i, log_scale=True)

In [ ]:
def R2(y, y_hat):
    y = np.array(y)
    y_hat = np.array(y_hat)
    SSE = np.sum((y - y_hat)**2)
    TSS = np.sum((y - np.mean(y))**2)
    R2 = 1 - SSE/TSS
    return R2

In [ ]:
R2(y, y_pred_type_only)

In [ ]:
R2(y, y_pred_mean)

In [ ]:
R2(y, y_pred)

Interpretation:

- For posts with no comments in the first hour, we expect 7.23 comments in total
- For every comment in the first hour, we expect an additional 6.43 comments in total

----

## Residuals

In [ ]:
plot_residuals(np.array(y), np.array(y_pred))

In [ ]:
np.unique(p_types, return_counts=True)

----

Generate posterior predictive samples

In [ ]:
svi_samples = get_samples(
    model, 
    guide, 
    p_data_train, 
    t_data, 
    s_data, 
    r_data, 
    None, 
    p_types_train, 
    p_stories_train,
    p_subreddits_train,
    [False],
    num_samples=100,
)

In [ ]:
inf_data = gather_az_inference_data(svi_samples, y_train)

---

Posterior distribution over latent variables:

In [ ]:
az.plot_trace(inf_data.posterior['phi'], compact=False)
plt.show()

---

Posterior predictive checks

In [ ]:
plot_pp_pdf(inf_data, y)

In [ ]:
plot_pp_cdf(inf_data, y)

- Max value

In [ ]:
plot_ppc(svi_samples, y_train, lambda x: np.max(x, axis=0), "Max", log_stats = True)

In [ ]:
plot_ppc(svi_samples, y_train, lambda x: np.mean(x, axis=0), "Mean", log_stats = True)

In [ ]:
plot_ppc(svi_samples, y_train, lambda x: np.var(x, axis=0), "Variance", log_stats = True)

In [ ]:
plot_ppc(svi_samples, y_train, lambda x: np.percentile(x, 90, axis=0), "90th %-ile (num obs)", log_stats = False)

In [ ]:
plot_ppc(svi_samples, y_train, lambda x: np.percentile(x, 99, axis=0), "99th %-ile (num obs)", log_stats = False)

In [ ]:
plot_ppc(svi_samples, y_train, lambda x: (x==0).mean(axis=0), "% = 0", log_stats = False)

In [ ]:
plot_ppc(svi_samples, y_train, lambda x: (x<10).mean(axis=0), "% < 10", log_stats = False)

In [ ]:
plot_ppc(svi_samples, y_train, lambda x: (x>1000).mean(axis=0), "% > 1000", log_stats = False)

# Why Pareto?
Our distribution of total comments follows a power law.

In [ ]:
max_ct = np.max(y)
unique, counts = np.unique(y + 1, return_counts=True)
plt.scatter(unique, counts)
plt.yscale("log")
plt.xscale("log")

plt.xlabel("total comments")
plt.ylabel("frequency")
plt.show()

Fake news: expect relatively flatter (more with lots, fewer with few!)

Reviews: expect steeper (fewer comments very common)

Note that our first hour comments are also distributed similarly.

In [ ]:
max_ct = np.max(p_data[:,1])
unique, counts = np.unique(p_data[:,1] + 1, return_counts=True)
plt.scatter(unique, counts)
plt.yscale("log")
plt.xscale("log")
#plt.xlim(1, 10000)
plt.xlabel("first hour comments")
plt.ylabel("frequency")
plt.show()

---